## 1. Consultas sobre redis (local)

In [15]:
# Conexion a redis 
import redis 
from redis.sentinel import Sentinel
from typing import Tuple, Any

# Lista de Sentinels
SENTINELS = [
    ('localhost', 26379),
    ('localhost', 26380),
    ('localhost', 26381)
]

# Creamos el objeto sentiels y vemos quien es el maestro 
GROUP_NAME = 'mymaster'  
sentinel = Sentinel(SENTINELS, socket_timeout=0.5, retry_on_timeout=True)
ip_network, port_network = sentinel.discover_master(GROUP_NAME)

'''
Diccionario de traduccion de ip de la subred.
Uso:
    - traduccion[ip_subred] = puerto_correspondiente
'''
traduccion = {
    "redis-master": 6379,
    "redis-replica1": 6380,
    "redis-replica2": 6381,
    "redis-replica3": 6382
}

MASTER_PORT = traduccion[ip_network]

r = redis.Redis(host='localhost',
                port=MASTER_PORT,
                decode_responses=True)

print(f"Contenedor actual actuando como master: {ip_network}")

Contenedor actual actuando como master: redis-master


In [16]:
# Para comprobar cuanta memoria usada, habria que utilizarla tanto por maestro como replicas  
from typing import Dict, Any

mem: Dict[str, Any] = r.info("memory") #type: ignore
used_mb = mem["used_memory"] / 1024 / 1024
total_mb = mem.get("total_system_memory", 0) / 1024 / 1024

print(f"Usada: {used_mb:.2f} MB")
print(f"Total sistema: {total_mb:.2f} MB")

Usada: 1.52 MB
Total sistema: 7567.27 MB


### 1.1 Colas de pacientes

Se guardan listas con prioridad de pacientes para atenderlos.

In [17]:
# Scan de las colas de pacientes
zsets = r.scan(match="ED-*:DEP-*", #type: ignore
       _type="ZSET")[1]
for key in zsets:
       print(key)

ED-5:DEP-PEDIATRIA
ED-3:DEP-OFTALMOLOGIA
ED-4:DEP-NEUROLOGIA
ED-2:DEP-NEUROLOGIA
ED-0:DEP-OFTALMOLOGIA


In [18]:
# Comprobar el contenido de las colas añadidas:
for key in zsets:
    print(f"Contenido de: {key}")
    scan_results = r.zscan(key,match="*")
    print(f"  Database --> {scan_results[0]}",end="\n    ") #type: ignore
    for patient in scan_results[1]: #type: ignore
        print(patient, end=" ")
    print("\n")

Contenido de: ED-5:DEP-PEDIATRIA
  Database --> 0
    ('Mariana', 5.0) ('Martina', 14.0) ('Valentina', 21.0) ('Andrés', 35.0) ('Mateo', 40.0) ('Daniel', 89.0) ('Sofía', 92.0) ('Alejandro', 97.0) 

Contenido de: ED-3:DEP-OFTALMOLOGIA
  Database --> 0
    ('Sofía', 4.0) ('Alejandro', 6.0) ('Camila', 24.0) ('Diego', 24.0) ('Javier', 34.0) ('Lucía', 43.0) ('Mateo', 45.0) ('Andrés', 100.0) 

Contenido de: ED-4:DEP-NEUROLOGIA
  Database --> 0
    ('Andrés', 36.0) ('Isabella', 56.0) ('Camila', 63.0) ('Alejandro', 73.0) ('Diego', 78.0) ('Martina', 91.0) ('Lucas', 94.0) ('Sebastián', 97.0) 

Contenido de: ED-2:DEP-NEUROLOGIA
  Database --> 0
    ('Diego', 16.0) ('Lucas', 21.0) ('Mateo', 24.0) ('Alejandro', 26.0) ('Javier', 46.0) ('Isabella', 54.0) ('Daniel', 67.0) ('Valentina', 89.0) 

Contenido de: ED-0:DEP-OFTALMOLOGIA
  Database --> 0
    ('Sofía', 22.0) ('Camila', 28.0) ('Diego', 35.0) ('Sebastián', 59.0) ('Andrés', 67.0) ('Javier', 78.0) ('Mariana', 80.0) ('Valentina', 80.0) 



In [19]:
# Pacientes prioritarios en cada cola 
for key in zsets: 
    print(key)
    [(patient, score)] = r.zrevrange(key, start=0, end=0, withscores=True) #type: ignore
    print(f"-> Paciente: {patient} | Prioridad: {score}")

ED-5:DEP-PEDIATRIA
-> Paciente: Alejandro | Prioridad: 97.0
ED-3:DEP-OFTALMOLOGIA
-> Paciente: Andrés | Prioridad: 100.0
ED-4:DEP-NEUROLOGIA
-> Paciente: Sebastián | Prioridad: 97.0
ED-2:DEP-NEUROLOGIA
-> Paciente: Valentina | Prioridad: 89.0
ED-0:DEP-OFTALMOLOGIA
-> Paciente: Valentina | Prioridad: 80.0


### 1.2 Alertas de salas

En las salas habra una dinamica pub/sub para que cuando suceda algo de relevancia en la sala (generalmente relacionado con problemas) se notifique instantaneamente.

Por la tematica pub/sub, la ingesta es dinamica, por lo que se incluira directamente aqui en el notebook. La idea es que se lanzara un hilo que publique alertas mientras el hilo principal se queda a la escucha. 

**NOTA**: Para detener la ejecicion, pulsar el boton de detener celda (en mi vscode se encuentra en la esquiza superior izquierda mientras tengas la celda seleccionada).

In [26]:
import threading
import time
import random
from typing import List

# Numeros de salas 
salas = [random.randint(0,20) for i in range(8)]

# Numeros de edificios 
edificios = [random.randint(0,5) for i in range(8)]

# Canales
# Formato de clave: ALERTAS:EDIFICIO:SALA
canales = list(zip(edificios, salas))
canales = [f"ALERTAS:ED-{edificio}:SALA-{sala}" for (edificio, sala) in canales]

# Funcion de ejemplo de publisher    
def publisher(canales: List[str]):

    factores = ["Temperatura", "Frecuencia cardiaca", "Glucosa en sangre",
                        "Frecuencia respiratoria", "Diuresis", "Nivel de conciencia"]

    posibles_alertas = ["Alta", "Baja"]

    # Vamos a meter 2 alertas por canal
    for canal in canales:
        for i in range(2):
            alerta = f"{factores[random.randint(0, len(factores))-1]}: {posibles_alertas[random.randint(0, len(posibles_alertas)-1)]}"
            r.publish(canal, alerta)
            time.sleep(0.5)
    print("[PUB] Todas las alertas enviadas!")

# Hilo principal: Subscriber
try:
    # Creamos el objeto tipo pub sub
    sub = r.pubsub()
    # Nos suscribimos a todos los canales
    for canal in canales:
        sub.subscribe(canal)
        
    # Creamos el hilo del publisher
    pub_thread = threading.Thread(target=publisher, args=([canales]))
    pub_thread.start()
    
    # Ponemos al hilo principal a escuchar 
    for msg in sub.listen():
        if msg['type'] == "message":
            print(f"[SUB] {msg["channel"]}: '{msg["data"]}'")
except KeyboardInterrupt: # 
    print("Subscriber terminado.")


[SUB] ALERTAS:ED-0:SALA-19: 'Nivel de conciencia: Baja'
[SUB] ALERTAS:ED-0:SALA-19: 'Frecuencia respiratoria: Alta'
[SUB] ALERTAS:ED-1:SALA-8: 'Frecuencia respiratoria: Alta'
[SUB] ALERTAS:ED-1:SALA-8: 'Diuresis: Alta'
[SUB] ALERTAS:ED-2:SALA-20: 'Frecuencia respiratoria: Alta'
[SUB] ALERTAS:ED-2:SALA-20: 'Frecuencia respiratoria: Alta'
[SUB] ALERTAS:ED-3:SALA-8: 'Nivel de conciencia: Baja'
[SUB] ALERTAS:ED-3:SALA-8: 'Frecuencia respiratoria: Alta'
[SUB] ALERTAS:ED-2:SALA-11: 'Nivel de conciencia: Alta'
[SUB] ALERTAS:ED-2:SALA-11: 'Frecuencia cardiaca: Alta'
[SUB] ALERTAS:ED-2:SALA-9: 'Frecuencia respiratoria: Alta'
[SUB] ALERTAS:ED-2:SALA-9: 'Frecuencia cardiaca: Baja'
[SUB] ALERTAS:ED-2:SALA-1: 'Diuresis: Baja'
[SUB] ALERTAS:ED-2:SALA-1: 'Nivel de conciencia: Baja'
[SUB] ALERTAS:ED-5:SALA-15: 'Frecuencia cardiaca: Alta'
[SUB] ALERTAS:ED-5:SALA-15: 'Temperatura: Alta'
[PUB] Todas las alertas enviadas!
Subscriber terminado.


### 1.3 Empleados

In [24]:
# Celda 1: Conexión
import redis
import json
from typing import Dict, Any

# Nos conectamos directamente a la DB=2 para empleados
r_empleados = redis.Redis(host='localhost',
                          port=MASTER_PORT,
                          db=2, # Conectamos a la DB=2
                          decode_responses=True)

# Verificamos la conexión
print(r_empleados.ping())

True


In [25]:
# Celda 2: Comprobar memoria y claves insertadas

# Comprobamos la memoria usada en esta DB
info = r_empleados.info('memory')
print(f"Memoria usada en DB 2: {info['used_memory_human']}")

# Buscamos las claves de nuestros empleados
print("\nClaves de empleados encontradas:")
# SCAN es más seguro que KEYS en producción
for key in r_empleados.scan_iter(match="hospital:madrid:empleado:*", count=10):
    print(key)

Memoria usada en DB 2: 1.43M

Claves de empleados encontradas:


In [26]:
# Celda 3: Obtener el JSON completo de un empleado

# Obtenemos todos los datos del empleado 1234
key_empleado = "hospital:madrid:empleado:1234"

# Usamos .json().get() para traer el objeto completo
# Es el equivalente a JSON.GET hospital:madrid:empleado:1234 $
empleado = r_empleados.json().get(key_empleado)

print(f"Datos del empleado {key_empleado}:")
# Usamos json.dumps para mostrarlo formateado
print(json.dumps(empleado, indent=2, ensure_ascii=False))

ResponseError: unknown command 'JSON.GET', with args beginning with: 'hospital:madrid:empleado:1234' '.' 

In [23]:
# Celda 4: Obtener campos específicos (Usando JSON Path)

key_empleado = "hospital:madrid:empleado:1234"
key_empleado_2 = "hospital:madrid:empleado:1236"

# Para obtener solo el puesto, usamos la ruta '$.puesto'
# El $ es la raíz, el .puesto accede al campo
puesto = r_empleados.json().get(key_empleado, "$.puesto")
print(f"Puesto del empleado 1234: {puesto[0]}") # Accedemos al primer elemento [0]

# Podemos pedir múltiples campos
datos_cardiologo = r_empleados.json().get(key_empleado_2, "$.nombre", "$.departamento", "$.salario")
print(f"Datos del cardiólogo (1236): {datos_cardiologo}")

ResponseError: unknown command 'JSON.GET', with args beginning with: 'hospital:madrid:empleado:1234' '$.puesto' 

In [ ]:
# Celda 5: Actualizar datos de un empleado

key_enfermero = "hospital:madrid:empleado:1235"

# 1. Le damos un aumento al enfermero Carlos Ruiz
# Usamos JSON.NUMINCRBY para sumar 2500 a su salario [cite: 2140]
print(f"Aumentando salario del empleado 1235...")
r_empleados.json().numincrby(key_enfermero, "$.salario", 2500)

# 2. Le cambiamos de sala
# Usamos JSON.SET para actualizar un campo existente [cite: 2133]
print(f"Cambiando de sala al empleado 1235...")
r_empleados.json().set(key_enfermero, "$.sala_asignada", "UCI Planta 5 - Box 3")

# 3. Verificamos los cambios
print("\n--- Datos actualizados del empleado 1235 ---")
datos_actualizados = r_empleados.json().get(key_enfermero, "$.nombre", "$.salario", "$.sala_asignada")
print(json.dumps(datos_actualizados, indent=2, ensure_ascii=False))

Aumentando salario del empleado 1235...
Cambiando de sala al empleado 1235...

--- Datos actualizados del empleado 1235 ---
{
  "$.salario": [
    44500
  ],
  "$.nombre": [
    "Enfermero Carlos Ruiz"
  ],
  "$.sala_asignada": [
    "UCI Planta 5 - Box 3"
  ]
}


## 2. Consultas sobre redis cloud 